In [0]:
import re
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import OrderedDict
from itertools import chain
import collections

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train = pd.read_csv('drive/My Drive/data_PA3/train.csv')
valid= pd.read_csv('drive/My Drive/data_PA3/valid.csv')
test = pd.read_csv('drive/My Drive/data_PA3/partial_test_400.csv')

In [4]:
#finding volcabulary size for English from the training data
train_eng = train["ENG"]
train_eng = np.array(train_eng)
print(train_eng.shape)

p = list(OrderedDict.fromkeys(chain.from_iterable(train_eng)))
input_vocab_size = len(p)+1
print(input_vocab_size)
print(p)

valid_eng = valid["ENG"]
valid_eng = np.array(valid_eng)
print(valid_eng.shape)

(13122,)
45
['R', ' ', 'A', 'S', 'V', 'I', 'H', 'E', 'D', 'O', 'G', 'N', '_', 'T', 'U', 'M', 'J', 'B', 'L', 'C', 'Y', 'K', 'Z', 'F', '-', 'W', 'P', 'Q', "'", 'X', '^', '/', '6', '(', ')', ',', 'É', '2', '.', '?', '4', 'Á', 'È', '1']
(997,)


In [5]:
#finding volcaublary size for Hindi from the training data
train_hin = train["HIN"]
# print(train_temp)
train_hin = np.array(train_hin)
print(train_hin.shape)

p = list(OrderedDict.fromkeys(chain.from_iterable(train_hin)))
output_vocab_size = len(p)+1
print(output_vocab_size)
print(p)

valid_hin = valid["HIN"]
valid_hin = np.array(valid_hin)

(13122,)
85
['र', ' ', 'ा', 'स', 'व', 'ि', 'ह', 'ी', 'द', 'े', 'ग', 'न', '_', 'ो', 'ड', 'श', 'त', '्', 'ु', 'म', 'ज', 'ब', 'ै', 'ल', 'क', 'ॉ', '़', 'य', 'फ', 'ट', 'इ', 'ज़', 'ऑ', 'ं', 'थ', '-', 'ू', 'अ', 'ध', 'प', '\u200d', 'छ', 'च', 'औ', 'ई', 'ॅ', 'आ', 'ख', 'ढ', 'ढ़', 'उ', 'झ', 'ँ', 'भ', 'ौ', 'ष', 'ण', 'घ', 'ठ', 'ए', 'ृ', 'क़', 'ऋ', 'ओ', 'ऐ', '/', 'ड़', '6', '(', ')', ',', 'ञ', "'", '2', '.', 'ऊ', '?', 'फ़', ':', '4', 'ख़', 'ग़', '1', '॥']


building encoder


In [6]:
inembsize = 256
outembsize = 256
batch_size = 20
num_batches = len(train)//batch_size

X_MAX_LENGTH = (max([len(train_eng[_]) for _ in range(len(train_eng))]))//2+1
Y_MAX_LENGTH = (max([len(train_hin[_]) for _ in range(len(train_hin))]))//2+1

num_epochs = 10
state_size = 512  #64, 128, 256 was a good number for linux OS!
encoder_layers = 1
decoder_layers = 1
learning_rate = 0.001
learning_rate_decay = 0.1

encoder_x = tf.placeholder(dtype=tf.int32, shape=[None, None]) #[batch_size, X_MAX_LENGTH]
decoder_x = tf.placeholder(dtype=tf.int32, shape=[None, None]) #[batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE]
y = tf.placeholder(dtype=tf.float32, shape=[None, None, None])#[batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE]
#init_state = tf.placeholder(tf.float32, [encoder_layers, 2, batch_size, state_size])

print(X_MAX_LENGTH)
print(Y_MAX_LENGTH)

61
62


In [0]:
def vectorize_data(batch_word, max_length, word_to_idx):
    vec_word = np.zeros((len(batch_word), max_length, len(word_to_idx)), dtype=float)
    for i, word in enumerate(batch_word):
        for j, char in enumerate(word):
            vec_word[i, j, char] = 1.
    return vec_word

In [0]:
def generate_text(prediction, batch_size, length, vocab_size, idx_to_word):

    batch_softmax = np.reshape(prediction, [batch_size, length, vocab_size])
    batch_word = []

    for word in batch_softmax:
        new_word = ''
        for char in word:
            vector_position = np.argmax(char)
            y_word = idx_to_word[vector_position]
            if y_word != '#':
                new_word = new_word + y_word + ' '
            else:
                new_word = new_word + ''
        batch_word.append(new_word)

    return batch_word


In [0]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [0]:
def load_data(train_eng, train_hin, vocabulary_size = 100):
    


    # Feed input data in backwards for better translation performance.
    x_data = [(tf.compat.as_str(train_eng[_]).lower().split(' ')) for _ in range(len(train_eng))]
    x_data = [list(x_data[_]) for _ in range(len(x_data))]
    X_DATA_SIZE = len(x_data)
    print(x_data[0:2])

    y_data = [(tf.compat.as_str(train_hin[_]).lower().split(' ')) for _ in range(len(train_hin))]
    y_data = [list(y_data[_]) for _ in range(len(y_data))]
    Y_DATA_SIZE = len(y_data)

    print(y_data[0:2])

    #Word Dictionary
    x_word_list = []
    y_word_list = []
    [x_word_list.extend(x_data[_]) for _ in range(len(x_data))]
    [y_word_list.extend(y_data[_]) for _ in range(len(y_data))]

    x_word_dictionary = []
    y_word_dictionary = []
    x_word_dictionary.extend(collections.Counter(x_word_list).most_common(44))
    y_word_dictionary.extend(collections.Counter(y_word_list).most_common(84))
    
    print(x_word_dictionary)

    #word/idx mapping
    x_idx_to_word = [word[0] for idx, word in enumerate(x_word_dictionary)]
    x_idx_to_word.insert(0, '#')
    x_idx_to_word.append('$')
    print(x_idx_to_word[0:20])

    y_idx_to_word = [word[0] for idx, word in enumerate(y_word_dictionary)]
    y_idx_to_word.insert(0, '#') #at the first 
    y_idx_to_word.append('$') #at the last
    print(y_idx_to_word[0:20])

    x_word_to_idx = {word:ix for ix, word in enumerate(x_idx_to_word)}
    y_word_to_idx = {word: ix for ix, word in enumerate(y_idx_to_word)}

    X_VOCAB_SIZE = len(x_word_dictionary) + 2
    Y_VOCAB_SIZE = len(y_word_dictionary) + 2

    X_MAX_LENGTH = max([len(x_data[_]) for _ in range(len(x_data))])
    Y_MAX_LENGTH = max([len(y_data[_]) for _ in range(len(y_data))])

    return X_VOCAB_SIZE, Y_VOCAB_SIZE, x_idx_to_word, x_word_to_idx, y_idx_to_word, y_word_to_idx, X_MAX_LENGTH, Y_MAX_LENGTH, X_DATA_SIZE


In [11]:
X_VOCAB_SIZE, Y_VOCAB_SIZE, x_idx_to_word, x_word_to_idx, y_idx_to_word, y_word_to_idx, X_MAX_LENGTH, Y_MAX_LENGTH, X_DATA_SIZE = load_data(train_eng, train_hin
                                                                                                                                            , 100)

[['r', 'a', 'a', 's', 'a', 'v', 'i', 'h', 'a', 'a', 'r', 'e', 'e'], ['d', 'e', 'o', 'g', 'a', 'n', '_', 'r', 'o', 'a', 'd']]
[['र', 'ा', 'स', 'व', 'ि', 'ह', 'ा', 'र', 'ी'], ['द', 'े', 'व', 'ग', 'न', '_', 'र', 'ो', 'ड']]
[('a', 22197), ('r', 9367), ('i', 8878), ('n', 8567), ('e', 8551), ('_', 6640), ('h', 6617), ('s', 5950), ('o', 5722), ('t', 5468), ('l', 5384), ('d', 4675), ('m', 4563), ('u', 4170), ('k', 3049), ('b', 2825), ('c', 2641), ('g', 2586), ('p', 2208), ('y', 2004), ('v', 1470), ('j', 1460), ('w', 1337), ('f', 1293), ('z', 545), ('q', 240), ('x', 136), ('-', 100), ("'", 58), (',', 32), ('.', 8), ('é', 7), ('^', 6), ('(', 6), (')', 6), ('/', 4), ('6', 2), ('2', 2), ('è', 2), ('?', 1), ('4', 1), ('á', 1), ('1', 1)]
['#', 'a', 'r', 'i', 'n', 'e', '_', 'h', 's', 'o', 't', 'l', 'd', 'm', 'u', 'k', 'b', 'c', 'g', 'p']
['#', 'ा', 'र', '्', '_', 'न', 'ि', 'ल', 'स', 'म', 'े', 'क', 'ी', 'व', 'ट', 'ो', 'ं', 'य', 'ब', 'द']


In [0]:
def get_data(x_word_to_idx, y_word_to_idx, inputs = train_eng, labels = train_hin):


    x_data = inputs
    y_data = labels

    #Feed input data in backwards for better translation performance.
    x_data = [(tf.compat.as_str(x_data[_]).lower().split(' ')) for _ in range(len(x_data))]
    x_data = [list(x_data[_]) for _ in range(len(x_data))]
    #x_data = x_data[start_batch:end_batch]
    
    
    y_data = [(tf.compat.as_str(y_data[_]).lower().split(' ')) for _ in range(len(y_data))]
    y_data = [list(y_data[_]) for _ in range(len(y_data))]
    #y_data = y_data[start_batch:end_batch]
    
    # Converting each character to its index value
    for i, word in enumerate(x_data):
        for j, charac in enumerate(word):
            if charac in x_word_to_idx:
                x_data[i][j] = x_word_to_idx[charac]
            else:
                x_data[i][j] = x_word_to_idx['$']

    for i, word in enumerate(y_data):
        for j, charac in enumerate(word):
            if charac in y_word_to_idx:
                y_data[i][j] = y_word_to_idx[charac]
            else:
                y_data[i][j] = y_word_to_idx['$']
                
    for _ in range(len(x_data)):
      x_length = len(x_data[_])
      y_length = len(y_data[_])

      x_data[_].extend(np.zeros([X_MAX_LENGTH - x_length], dtype=int))
      y_data[_].extend(np.zeros([Y_MAX_LENGTH - y_length], dtype=int))

    return x_data, y_data

In [0]:
restore = False
ckpt_model_directory = 'drive/My Drive/data_files/training/saved_models'

In [0]:
x_data , y_data = get_data( x_word_to_idx, y_word_to_idx, inputs = train_eng, labels = train_hin)                                              

In [0]:
x_data_valid , y_data_valid = get_data(x_word_to_idx, y_word_to_idx, inputs = valid_eng, labels = valid_hin)

In [17]:
with tf.variable_scope('encoder_word_embeddings'):

    word_embeddings = tf.get_variable('encoder_word_embeddings', [X_VOCAB_SIZE, inembsize])
    encoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, encoder_x)
    encoder_embedded_word_ids = tf.reshape(encoder_embedded_word_ids, [-1, X_MAX_LENGTH, inembsize])

with tf.variable_scope('encoder'):

#             state_per_layer_list = tf.unstack(init_state, axis=0)
#             rnn_tuple_state = tuple(
#                 [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])
#                 for idx in range(encoder_layers)])

#             encoder_stacked_cell = []

#             for _ in range(encoder_layers):
#                 encoder_single_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
#                 if _ == 0:
#                     with tf.name_scope('encoder_dropout') as scope:
#                         encoder_single_cell = tf.nn.rnn_cell.DropoutWrapper(encoder_single_cell,
#                                                         output_keep_prob=0.75)  # add dropout to first LSTM layer only.
#                 encoder_stacked_cell.append(encoder_single_cell)

#             encoder_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_stacked_cell, state_is_tuple=True)

#             encoder_outputs, encoder_state = tf.nn.dynamic_rnn(cell=encoder_cell,
#                                                              inputs=encoder_embedded_word_ids,
#                                                              dtype = tf.float32)

            #del encoder_cell, state_per_layer_list, encoder_stacked_cell, rnn_tuple_state

    for n in range(encoder_layers):
          (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
              cell_fw = tf.nn.rnn_cell.LSTMCell(state_size//2, state_is_tuple=True),  ## initializer, reuse to be set
              cell_bw = tf.nn.rnn_cell.LSTMCell(state_size//2, state_is_tuple=True),  ## initializer, reuse to be set
              inputs = encoder_embedded_word_ids,
              
              sequence_length =  [61]* batch_size,
              dtype = tf.float32)

          encoder_embedded_word_ids = tf.concat((out_fw, out_bw), 2)

          bi_state_c = tf.concat((state_fw.c, state_bw.c), -1)
          bi_state_h = tf.concat((state_fw.h, state_bw.h), -1)
          bi_lstm_state = tf.nn.rnn_cell.LSTMStateTuple(c=bi_state_c, h=bi_state_h)
          encoder_state = tuple([bi_lstm_state] * encoder_layers)

          encoder_state = tuple(encoder_state[-1] for _ in range(encoder_layers))

with tf.variable_scope('decoder_word_embeddings'):

    word_embeddings = tf.get_variable('decoder_word_embeddings', [Y_VOCAB_SIZE, outembsize])
    decoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, decoder_x)
    #decoder_embedded_word_ids = tf.reshape(decoder_embedded_word_ids,
                               #                 [-1, Y_MAX_LENGTH,  outembsize])


with tf.variable_scope('decoder'):

#             if decoder_layers != 0:
#               null_state = np.zeros((decoder_layers-1, 2,  batch_size, state_size))
#               null_state = tf.convert_to_tensor(null_state,dtype=tf.float32)
#               encoder_final_state = tf.concat([encoder_final_state, null_state], 0)

#               state_per_layer_list1 = tf.unstack(encoder_final_state, axis=0)
#               encoder_final_state = tuple(
#                 [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list1[idx][0], state_per_layer_list1[idx][1])
#                 for idx in range(decoder_layers)])  


#             decoder_stacked_cell = []
#             for _ in range(decoder_layers):
#                 decoder_single_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)  ## initializer, reuse to be set
#                 if _ == 0:
#                     with tf.name_scope('decoder_dropout') as scope:
#                         decoder_single_cell = tf.nn.rnn_cell.DropoutWrapper(decoder_single_cell,
#                                                     output_keep_prob=0.75)  
#                 decoder_stacked_cell.append(decoder_single_cell)

#             decoder_cell = tf.nn.rnn_cell.MultiRNNCell(decoder_stacked_cell, state_is_tuple=True)

#             decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(cell= decoder_cell,
#                                                              inputs=decoder_embedded_word_ids,
#                                                            initial_state=encoder_final_state)

    decoder_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(state_size) for _ in range(decoder_layers)])

    dense_layer = tf.layers.Dense(Y_VOCAB_SIZE)

    #decoder for training data
    training_helper = tf.contrib.seq2seq.TrainingHelper(
            inputs = decoder_embedded_word_ids,
            sequence_length = [62]* batch_size)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(
            cell = decoder_cell,
            helper = training_helper,
            initial_state = encoder_state,
            output_layer = dense_layer)

    training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
            decoder = training_decoder,
            impute_finished = True,   ## impute finished make sure that weights causing correctly predicted words are not changed further
            maximum_iterations = Y_MAX_LENGTH)   

    #decoder for test data
#             predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
#                     embedding = decoder_embedded_word_ids,
#                     start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
#                     end_token = EOS)

#             predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
#                     cell = decoder_cell,
#                     helper = predicting_helper,
#                     initial_state = encoder_state,
#                     output_layer = dense_layer)

#             predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
#                     decoder = predicting_decoder,
#                     impute_finished = True,
#                     maximum_iterations = 2 * (X_MAX_LENGTH))


    training_logits = training_decoder_output.rnn_output
    softmax_out = tf.nn.softmax(training_logits)

    #predicting_ids = predicting_decoder_output.sample_id
    #masks = tf.sequence_mask(tf.count_nonzero(decoder_x, 1, dtype=tf.int32), Y_MAX_LENGTH, dtype=tf.float32)  

    #cost = tf.contrib.seq2seq.sequence_loss(logits = softmax_out,targets = decoder_x)
                                                                                        
    cost =  tf.nn.softmax_cross_entropy_with_logits(logits=training_logits, labels=y)
    
    total_cost = tf.reduce_mean(cost)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_cost)

    y_t = tf.argmax(softmax_out,axis=2)
    y_t = tf.cast(y_t, tf.int32)

    #prediction = tf.boolean_mask(y_t, masks)
    prediction = y_t
    #mask_label = tf.boolean_mask(decoder_x, masks)
    mask_label = decoder_x
    
    correct_pred = tf.equal(prediction, mask_label)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    saver = tf.train.Saver(max_to_keep=30)
    #outputs = tf.reshape(decoder_outputs, [-1, state_size])

#             with tf.name_scope('decoder_hidden_states') as scope:
#                 W2 = tf.Variable(tf.random_normal([state_size, output_vocab_size]), dtype=tf.float32)
#                 variable_summaries(W2)
#                 b2 = tf.Variable(tf.zeros([1, output_vocab_size]), dtype=tf.float32)
#                 variable_summaries(b2)

#                 logits = tf.matmul(outputs, W2) + b2 # Broadcasted addition
#                 tf.summary.histogram('pre_activations', logits)
#                 prediction = tf.nn.softmax(logits)
#                 tf.summary.histogram('activations', prediction)
#             labels = y

#         with tf.name_scope('cross_entropy') as scope:
#             cost = tf.contrib.seq2seq.sequence_loss(logits = training_logits,
#                                                      targets = y,
#                                                      weights = masks)
#             total_cost = tf.reduce_mean(cost)
#             tf.summary.scalar('cross-entropy', total_cost)

#         with tf.name_scope('optimizer') as scope:
#             optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_cost)

#         #Keep previous 30 model checkpoints..


#return encoder_state, labels , training_logits, prediction



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_

In [18]:
with tf.Session() as sess:

    merged = tf.summary.merge_all()

    sess.run(tf.global_variables_initializer())

    epoch = 0

#         if restore:
#             print('Loading variables from {}'.format(ckpt_model_directory + nn_config.meta_file + nn_config.meta_num + nn_config.meta_ext))
#             saver.restore(sess, nn_config.ckpt_model_directory + nn_config.meta_file + nn_config.meta_num)


    #load data
    while epoch<1: 
        epoch_loss = 0

        for batch in range(num_batches):
            start_batch = batch * batch_size
            end_batch = start_batch + batch_size

            batch_loss = 0

            batch_x = x_data[start_batch:end_batch]
            batch_y = y_data[start_batch:end_batch]


            y_one_hot = vectorize_data(batch_y, Y_MAX_LENGTH, y_word_to_idx)
            
            opt = sess.run(optimizer, feed_dict={encoder_x: batch_x,decoder_x: batch_y,y: y_one_hot})
            _total_cost, _accuracy, = sess.run([total_cost, accuracy], feed_dict={encoder_x: batch_x,decoder_x: batch_y,y: y_one_hot})     
            logits, _prediction_series = sess.run([training_logits, softmax_out], feed_dict={encoder_x: batch_x,decoder_x: batch_y,y: y_one_hot})     
            if batch % 1 == 0:

                text = generate_text(_prediction_series, batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE, y_idx_to_word)

                if epoch % 5 == 0:
                    save_output_path = 'drive/My Drive/data_PA3/TRAINING_TRANSLATION_OUTPUT_EPOCH_{}_STATE-SIZE_{}_NUM-LAYERS_{}_LEARNING-RATE{}_EMBEDDING-SIZE_{}.txt'.format(epoch, state_size, decoder_layers, learning_rate, inembsize)

                    with open(save_output_path,'w') as f:
                       for item in text:
                           f.write("%s\n" % item)
                    print('[NOTIFICATION] Translation file saved to: {}'.format(save_output_path))

                print('[INFO]: Batch {} optimized output for Epoch {}:'.format(batch, epoch))
                for _ in range(batch_size):
                    print(text[_])

            batch_loss += _total_cost
            epoch_loss += _total_cost

            print('\n[STATUS] Batch {}/{} complete! Batch Loss: {}'.format(batch, num_batches, batch_loss))
            print(_accuracy)
            print(batch)

        if epoch % 1 == 0:

            save_model_path = "drive/My Drive/data_PA3/"
            if epoch == 0:
                save_path = saver.save(sess, save_model_path, epoch)
            else:
                save_path = saver.save(sess, save_model_path, epoch, write_meta_graph=False)

            print("[STATUS] Model ckpt saved in file: %s" % save_path)

        print('[STATUS] Epoch {} complete! Epoch Loss: {}'.format(epoch, epoch_loss))

        saver.save(sess, save_model_path + '/model.ckpt', epoch)
        epoch+=1


[NOTIFICATION] Translation file saved to: drive/My Drive/data_PA3/TRAINING_TRANSLATION_OUTPUT_EPOCH_0_STATE-SIZE_512_NUM-LAYERS_1_LEARNING-RATE0.001_EMBEDDING-SIZE_256.txt
[INFO]: Batch 0 optimized output for Epoch 0:
् 
ा ा 
् 
् 
ा 

् 
् 
् ् 
् 
् ् 
् 
् ् 
् 
ा ा 
् 
ा 
ा 
् 
् ् 

[STATUS] Batch 0/656 complete! Batch Loss: 3.681623697280884
0.8516129
0
[NOTIFICATION] Translation file saved to: drive/My Drive/data_PA3/TRAINING_TRANSLATION_OUTPUT_EPOCH_0_STATE-SIZE_512_NUM-LAYERS_1_LEARNING-RATE0.001_EMBEDDING-SIZE_256.txt
[INFO]: Batch 1 optimized output for Epoch 0:



ा 





ा 


ा 
ा 
ा 

ा 

ा 


[STATUS] Batch 1/656 complete! Batch Loss: 1.965707540512085
0.8475807
1
[NOTIFICATION] Translation file saved to: drive/My Drive/data_PA3/TRAINING_TRANSLATION_OUTPUT_EPOCH_0_STATE-SIZE_512_NUM-LAYERS_1_LEARNING-RATE0.001_EMBEDDING-SIZE_256.txt
[INFO]: Batch 2 optimized output for Epoch 0:





















[STATUS] Batch 2/656 complete! Batch Loss: 0.8378551602363586
0.841129
2


KeyboardInterrupt: ignored

In [0]:
y_one_hot = vectorize_data(y_data, Y_MAX_LENGTH, y_word_to_idx)

In [0]:
x_data

In [0]:
save_model_path  = "drive/My Drive/data_PA3/"

In [0]:
with tf.Session() as sess:

    g = tf.get_default_graph()

    new_saver = tf.train.import_meta_graph(save_model_path + '-0' + '.meta')

    new_saver.restore(sess, tf.train.latest_checkpoint(save_model_path))
    
    _current_state = np.zeros((encoder_layers, 2,  batch_size, state_size))
    
    
    with tf.name_scope('decoder', reuse=tf.AUTO_REUSE) as scope:
      _prediction_series = sess.run(
              [ prediction],
              feed_dict={
                      encoder_x: x_data_valid,
                      decoder_x: y_data_valid,
                      y: y_one_hot,
                      init_state: _current_state
                  })